## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x107e0e7d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x107e0f490>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Susovan and I am an AI/ML Engineer")])

AIMessage(content="Hello Susovan,\n\nIt's nice to meet you! I'm glad you introduced yourself.\n\nAs an AI/ML Engineer, what kind of projects are you working on these days?  I'm always interested to hear about the cool things people are doing with AI and machine learning.\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 30, 'total_tokens': 95, 'completion_time': 0.133110463, 'prompt_time': 0.004100567, 'queue_time': None, 'total_time': 0.13721103}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1a1ecdea-f2e5-48e8-991d-e6aeae4c692f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 65, 'total_tokens': 95})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Susovan and I am an AI/ML Engineer"),
        AIMessage(content="Hello Susovan,\n\nIt's nice to meet you! I'm glad you introduced yourself.\n\nAs an AI/ML Engineer, what kind of projects are you working on these days?  I'm always interested to hear about the cool things people are doing with AI and machine learning.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You told me your name is Susovan and that you are an AI/ML Engineer. 😊  \n\nDo you have any other questions for me?  \n', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 115, 'total_tokens': 150, 'completion_time': 0.070867687, 'prompt_time': 0.006001267, 'queue_time': None, 'total_time': 0.076868954}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dfd56733-640b-4c57-bcc5-e2a223e12d63-0', usage_metadata={'input_tokens': 115, 'output_tokens': 35, 'total_tokens': 150})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , I am Susovan and I am an AI/ML Engineer")],
    config=config
)

In [9]:
response.content

"Hi Susovan, it's nice to meet you!\n\nThat's awesome, AI/ML Engineering is a really exciting field. \n\nWhat kind of projects are you working on these days? 🤖🧠  Anything you're particularly passionate about?\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='According to our conversation, your name is Susovan.  😊  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 100, 'total_tokens': 129, 'completion_time': 0.05763028, 'prompt_time': 0.008713252, 'queue_time': None, 'total_time': 0.066343532}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2693c498-4680-4744-bb9f-95fda14d3dd2-0', usage_metadata={'input_tokens': 100, 'output_tokens': 29, 'total_tokens': 129})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 😊\n\nIs there anything I can help you with today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember that you told me earlier.  😊  Is there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sanu")]})

AIMessage(content="Hello Sanu, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 37, 'total_tokens': 64, 'completion_time': 0.053412123, 'prompt_time': 0.006454155, 'queue_time': None, 'total_time': 0.059866277999999995}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6dbcbd9-efae-4186-9173-880bcda012fe-0', usage_metadata={'input_tokens': 37, 'output_tokens': 27, 'total_tokens': 64})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sanu")],
    config=config
)

response

AIMessage(content="Hello Sanu! 👋\n\nIt's nice to meet you too.  What can I do for you? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 78, 'total_tokens': 106, 'completion_time': 0.055554401, 'prompt_time': 0.012907288, 'queue_time': None, 'total_time': 0.068461689}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7528572d-955a-4796-99cc-f79837b6c0be-0', usage_metadata={'input_tokens': 78, 'output_tokens': 28, 'total_tokens': 106})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sanu. 😊  \n\nIs there anything else I can help you with?  \n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sanu")],"language":"Bengali"})
response.content

'নমস্কার, সানু! 😊  আপনার নাম কি সানু?  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Susovan")],"language":"Bengali"},
    config=config
)
repsonse.content

'নমস্কার সুসভান! 😊 \n\nআমি তোমার জন্য সাহায্য করার জন্য এখানে। কিছু প্রশ্ন থাকলে জিজ্ঞাসা করো, আমি যত পারি উত্তর দেব। \n\n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Bengali"},
    config=config,
)

In [26]:
response.content

'তোমার নাম সুসভান। 😉 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [27]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/susovanpatra/Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/susovanpatra/Langchain/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [28]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?🍦\n"

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What\'s 2 + 2?" 😊  \n\n\n\n\n'

In [30]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [31]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIs there anything else I can help you with? 😊\n"

In [32]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"